## Importing Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Preprocessing

In [82]:
path = "./drive/My Drive/data/"
train_path = path+"Train.csv"
test_path = path+"Test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGHGSN,1,1.086711e+04,8.648345e+03,1.576618e+04,2.890466e+04,5.273655e+03,2.346153e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,NaN,NaN,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1.414214,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,214.0,214.0,68.0,22.0,22.0,7.0
1,RIMHI5ZGP,0,2.584312e+05,2.179633e+05,1.866287e+05,1.866287e+05,2.154519e+05,1.923615e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,NaN,NaN,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,NaN,1,1,3,4,4,0,4,5,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.0,41.0,38.0,12.0,12.0,11.0
2,RIHOG1SZU,0,1.459355e+07,1.260060e+07,1.100315e+07,1.010315e+07,1.879882e+07,1.514035e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,NaN,NaN,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,NaN,3,3,4,12,4,0,2,4,NaN,1.0,NaN,NaN,...,1,0,1,0,1,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,378.0,378.0,310.0,21.0,21.0,16.0
3,RIHOLMQWU,0,6.727043e+05,8.181116e+05,6.794893e+05,6.794893e+05,8.257254e+05,7.605803e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,NaN,NaN,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,NaN,2,2,2,24,1,0,1,4,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1,2,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,0,702.0,702.0,644.0,951.0,951.0,743.0
4,RIHO584ET,0,-1.218742e+06,-3.005361e+06,-1.666241e+05,-2.403574e+06,2.583183e+06,2.086546e+06,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,NaN,4,3,4,68,8,0,2,4,NaN,3.0,NaN,NaN,...,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1305.0,1305.0,1211.0,2205.0,2205.0,1831.0


In [5]:
test_df.sample(5)

,Col1,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
14764,O1WWO85W,18516.44667,16424.85500,14078.44714,16424.85500,23423.89167,23423.89167,10524.549380,10524.549380,32101.39030,32101.39030,9.826415,10.061512,10.061512,9.706551,1.166667,1.291851,NaN,NaN,0.640770,1.370455,0.640770,1.370455,0.266762,0.392196,0.260310,NaN,2,2,2,2,1,1,0,5,NaN,NaN,NaN,NaN,3.0,...,0,0,0,0,0,0,0,0,0,0,2,1,2,3,0,1,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,19.0,19.0,11.0,12.0,12.0,5.0
7879,7N0HGOYW,14857.57667,18999.84833,18999.84833,37999.69667,14766.16667,29532.33333,27289.181390,27701.893320,19494.84059,18000.24767,9.606265,9.600094,10.293241,10.545333,1.000000,0.243002,NaN,NaN,1.436284,1.320237,0.729003,0.609510,0.106067,0.129619,0.199015,-0.765345,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.0,9.0,6.0,5.0,4.0,1.0
14720,O1WBIS4D,21610.29333,13576.41000,11636.92286,13576.41000,11458.22500,11458.22500,9145.072384,9145.072384,10797.31691,10797.31691,9.980925,9.346463,9.346463,9.516089,1.166667,3.898997,NaN,NaN,0.673600,0.942320,0.673600,0.942320,0.581395,0.563912,0.773859,NaN,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,NaN,...,0,0,1,2,0,0,0,0,0,0,3,1,1,1,0,0,0,0,3,1,3,1,0,0,0,0,0,0,0,0,1,2,1,2,22.0,22.0,13.0,9.0,9.0,4.0
3999,O1HWIMQE,93451.60333,77058.73500,68009.08571,77058.73500,55690.47167,55690.47167,18781.057830,18781.057830,25307.81936,25307.81936,11.445199,10.927564,10.927564,11.252323,1.133065,1.540431,5.620135,2.223439,0.243724,0.454437,0.243724,0.454437,0.549293,0.575648,0.759797,NaN,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,2,0,2,0,97.0,97.0,46.0,15.0,13.0,6.0
18473,O1YKLS55,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,1,1,1,1,1,1,0,4,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


1st column is ID and Second columns is Target variable

In [0]:
# Adding one because we will split at this point
train_last_index = 17521

In [26]:
train_df.shape, test_df.shape

((17521, 2395), (20442, 2394))

## TODO:Combine test train and process

In [27]:
X = pd.concat((train_df.iloc[:,2:],test_df.iloc[:,1:]),axis=0)

X.shape

(37963, 2393)

In [0]:
# Columns Null check
def null_cols(d,columns):
  na = d[columns].isna().sum().sort_values(ascending=False)
  return na[na > 0].index.to_list()

In [0]:
X.

In [30]:
# Numerical columns
num_cols = X.select_dtypes(include="number").columns
print('Total',len(num_cols))

na_cols = null_cols(X,num_cols)
print('1.Null columns',len(na_cols))

if len(na_cols) > 0:
  X[na_cols]= X[na_cols].fillna(method='ffill')
  X[na_cols]= X[na_cols].fillna(method='bfill')
  na_cols = null_cols(X,num_cols)
  print('2.Null columns',len(na_cols))

Total 2380
1.Null columns 746
2.Null columns 0


### Categorical columns

In [31]:
cat_cols = X.select_dtypes(exclude="number").columns.to_list()

print('Total',len(cat_cols))
na_cols = null_cols(X,cat_cols)
print('1.Null columns',len(na_cols))

if len(na_cols) > 0:
  X[na_cols]= X[na_cols].fillna(method='ffill')
  X[na_cols]= X[na_cols].fillna(method='bfill')
  na_cols = null_cols(X,cat_cols)
  print('2.Null columns',len(na_cols))

Total 13
1.Null columns 13
2.Null columns 0


In [42]:
for col in cat_cols:
  print('Before',X[col].value_counts())
  X[col] = X[col].replace({0.0:0,'-':2})
#   print('After',X[col].value_counts(),'\n')

Before 0.0                     37326
0                         231
0.058166302999999996      165
0.057142859000000004      103
0.255550626                97
0.07985957099999999        16
-                          11
0.44261957799999996         8
0.069985428                 6
Name: Col702, dtype: int64
After 0                       37326
0                         231
0.058166302999999996      165
0.057142859000000004      103
0.255550626                97
0.07985957099999999        16
2                          11
0.44261957799999996         8
0.069985428                 6
Name: Col702, dtype: int64 

Before 0                       9420
-                       3604
0.349927106             1518
0.447213596             1423
0.0                     1392
0.283473355             1060
-0.2                    1022
0.223606798              779
0.188982237              691
0.06998542099999999      578
0.6                      439
0.20995626399999998      417
0.2                      404
-0.0699

### Separating the Y variable

In [43]:
y = train_df.iloc[:,[1]]
print( y.sample(5))
y.shape

       Col2
5661      0
663       0
13012     0
4886      0
71        0


(17521, 1)

In [44]:
y.columns

Index(['Col2'], dtype='object')

In [45]:
null_cols(y,y.columns)

[]

### t-SNE


 n | divergence | time
---|---| ---
3| 1.552975 | 50 mins 

In [46]:
if X.isna().sum().sum() < 1:
  model = TSNE(n_components=3,random_state=0,verbose=1)
  tsne_data = model.fit_transform(X)
  np.save('tsne',tsne_data)
else:
  print('Null vals present')

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 37963 samples in 5.637s...
[t-SNE] Computed neighbors for 37963 samples in 986.562s...
[t-SNE] Computed conditional probabilities for sample 1000 / 37963
[t-SNE] Computed conditional probabilities for sample 2000 / 37963
[t-SNE] Computed conditional probabilities for sample 3000 / 37963
[t-SNE] Computed conditional probabilities for sample 4000 / 37963
[t-SNE] Computed conditional probabilities for sample 5000 / 37963
[t-SNE] Computed conditional probabilities for sample 6000 / 37963
[t-SNE] Computed conditional probabilities for sample 7000 / 37963
[t-SNE] Computed conditional probabilities for sample 8000 / 37963
[t-SNE] Computed conditional probabilities for sample 9000 / 37963
[t-SNE] Computed conditional probabilities for sample 10000 / 37963
[t-SNE] Computed conditional probabilities for sample 11000 / 37963
[t-SNE] Computed conditional probabilities for sample 12000 / 37963
[t-SNE] Computed conditional probabilities for s

In [47]:
tsne_data.shape

(37963, 3)

In [49]:
type(tsne_data)

numpy.ndarray

## EDA

In [0]:
# X_train and X_test are both without ID column
X_train = tsne_data[:train_last_index]
X_test = tsne_data[train_last_index:]

In [51]:
X_train[:10]

array([[ 12.966208 ,   7.6291747, -10.028655 ],
       [-14.196468 ,  21.274002 ,  -8.30545  ],
       [-17.39448  ,  20.732141 ,  -0.725202 ],
       [-15.47157  ,  21.346645 ,  -1.2622602],
       [-17.6458   ,  20.615833 ,  -0.7474113],
       [-15.011502 ,  21.37602  ,  -1.7775688],
       [  3.7999327,  -9.978249 ,   4.8382   ],
       [  3.3549275,   3.6714833, -10.211234 ],
       [  6.1283283,   1.5645589,   6.205583 ],
       [ 12.628237 ,  10.502278 , -15.479282 ]], dtype=float32)

In [52]:
X_train.shape

(17521, 3)

In [53]:
join_data = np.hstack((X_train,y))
print(join_data.shape)
dfa = pd.DataFrame(join_data,columns=("dim1",'dim2','dim3','label'))
dfa.head()

(17521, 4)


,dim1,dim2,dim3,label
0,12.966208,7.629175,-10.028655,1.0
1,-14.196468,21.274002,-8.305450,0.0
2,-17.394480,20.732141,-0.725202,0.0
3,-15.471570,21.346645,-1.262260,0.0
4,-17.645800,20.615833,-0.747411,0.0


In [57]:
dfa.describe()

,dim1,dim2,dim3,label
count,17521.000000,17521.000000,17521.000000,17521.000000
mean,-0.304923,-0.110430,-0.087895,0.100508
std,10.946952,11.210497,10.088150,0.300685
min,-23.939781,-24.082001,-21.857027,0.000000
25%,-9.103204,-9.647172,-8.245725,0.000000
50%,-0.146481,0.380391,-0.070401,0.000000
75%,8.429140,8.930604,8.053248,0.000000
max,25.324026,26.590508,23.967939,1.000000


### dim1 v/s dim2

In [0]:
ax = sns.FacetGrid(dfa,hue="label",height=6).map(sns.distplot,'dim1').add_legend()
ax1 = sns.FacetGrid(dfa,hue="label",height=6).map(sns.distplot,'dim2').add_legend()
ax3 = sns.FacetGrid(dfa,hue="label",height=6).map(sns.distplot,'dim3').add_legend()

## Modelling

In [59]:
X_train.shape, y.shape, X_test.shape

((17521, 3), (17521, 1), (20442, 3))

### Test data prep

In [60]:
test_id = test_df.iloc[:,0]
test_id.shape

(20442,)

In [62]:
test = pd.DataFrame(X_test,columns=["dim1",'dim2','dim3'])
test.describe()

,dim1,dim2,dim3
count,20442.000000,20442.000000,20442.000000
mean,0.344181,-0.077773,0.218283
std,10.500479,10.951061,10.244196
min,-24.032606,-24.129465,-21.797735
25%,-7.882465,-9.187041,-7.969855
50%,0.811122,0.172071,-0.026267
75%,8.608865,8.602738,8.531698
max,25.332592,26.505873,23.942247


In [0]:
# logistic baseline classification model
# accuracy: 0.8994920381256778
# Leaderboard: 84.03866
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='saga',multi_class='ovr').fit(X_train, y['Col2'])

In [0]:
# Random forest
# accuracy: 0.8994920381256778
# Leaderboard: 
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0).fit(X_train, y['Col2'])

In [129]:
# XGBoost
import xgboost as xgb

train_X_mat = np.matrix(X_train)
test_X_mat = np.matrix(X_test)

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X_mat, y)
predictions = gbm.predict(test_X_mat)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Decision tree
# accuracy: 0.9971462816049312
# Leaderboard: 81.30581
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state = 0).fit(X_train, y['Col2'])

## Evaluation

In [0]:
# Evaluate train accuracy
from sklearn.metrics import f1_score,accuracy_score

print(clf)
y1 = clf.predict(X_train)

# yTrue, yPred
# f1_score(y['Col2'],y1,average='binary')
accuracy_score(y['Col2'],y1)

In [135]:
accuracy_score(y['Col2'],predictions)

0.8994920381256778

In [120]:
sample = pd.read_csv(path+'Sample_submission.csv')
y_test = clf.predict(test)
print(sum(y_test))

2243


In [114]:
test_df.iloc[:,0].shape,y_test.shape

((20442,), (20442,))

In [0]:
sample.drop(index=sample.index,inplace=True)

In [0]:
sample['Col1'] = test_df.iloc[:,0]
sample['Col2'] = y_test 
sample.to_csv('out.csv',index=False)

## Can try

In [0]:
# Clean outlier in data
# Replacing numeric values with median and cat with mode
# Deciding the number of components for tsne
# Feature selection if ML
# grid search
# Random search
# neural network
# XGBoost 
# lightgbm
# Shuffle and fill values for second time
# 0   and 0.0 were different